In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from bloptools import test_functions

import os

os.environ[
    "PATH"
] = "/opt/homebrew/opt/llvm/bin:/Users/tom/opt/anaconda3/bin:/Users/tom/opt/anaconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/opt/X11/bin"

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
    }
)

cummax = lambda iterable: np.array([np.nanmax(iterable[:i]) for i in range(1, len(iterable) + 1)])


no_latent_paths = glob.glob("../data/benchmark_hartmann6_no_latent/*.h5")
print(len(no_latent_paths))
no_latent_table = pd.DataFrame()

for path in no_latent_paths:
    no_latent_table.loc[:, path] = -cummax(np.exp(pd.read_hdf(path, key="table").neg_hartmann_fitness.values))


latent_paths = sorted(glob.glob("../data/benchmark_hartmann6_latent-2/*.h5"))
print(len(latent_paths))
latent_table = pd.DataFrame()

for path in latent_paths:
    # print(path)
    latent_table.loc[:, path] = -cummax(np.exp(pd.read_hdf(path, key="table").neg_hartmann_fitness.values))


pd.read_hdf(path, key="table").acq_func.values[:10]

max_iter = 256

iter_bins = np.linspace(0, max_iter, 64)


import pandas as pd
import glob

fig, ax = plt.subplots(1, 1, figsize=(6, 6))


for col in no_latent_table.columns:
    loss = no_latent_table.loc[:, col].values
    ax.plot(np.arange(len(loss)) + 1, loss, lw=1e-1, c="r")

for col in latent_table.columns:
    loss = latent_table.loc[:, col].values
    ax.plot(np.arange(len(loss)) + 1, loss, lw=1e-1, c="b")

ax.plot(np.median(no_latent_table, axis=1), c="r", lw=1e0)
ax.plot(np.median(latent_table, axis=1), c="b", lw=1e0)

xmin, xmax = 0, max_iter
ymin, ymax = ax.get_ylim()

ax.set_xlabel("iteration")
ax.set_ylabel("cumulative maximum")

optimum_inputs = None
optimum_target = -3.32237

ax.plot([xmin, xmax], [optimum_target, optimum_target], ls=":", c="k")

ax.fill_betweenx([ymin, ymax], xmin, 64, alpha=0.1, label="quasi-random initialization")

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# ax.set_yscale("log")

plt.tight_layout()
plt.savefig("../plots/bayesian_himmelblau.pdf", bbox_inches="tight", transparent=True, pad_inches=0, dpi=256)

In [ ]:
RE(agent.learn("qei", n=2, iterations=16))

In [ ]:
np.round(agent.tasks[0]["model"].covar_module.latent_transform.detach(), 3)

In [ ]:
agent.plot_tasks()

In [ ]:
qei, _ = bloptools.bayesian.acquisition.get_acquisition_function(agent, "qei")

In [ ]:
qucb, _ = bloptools.bayesian.acquisition.get_acquisition_function(agent, "qucb")

In [ ]:
X, _ = agent.ask("qei", n=8)

In [ ]:
import torch

x1 = torch.linspace(-6, 6, 63)
x2 = torch.linspace(-6, 6, 63)
X1, X2 = torch.meshgrid(x1, x2, indexing="ij")

xg = torch.cat([X1.unsqueeze(-1), X2.unsqueeze(-1)], dim=-1)
obj_grid = qei(xg.reshape(-1, 1, 2)).reshape(xg.shape[:2]).detach()

In [ ]:
import scipy as sp

In [ ]:
optima = [(3, 2), (-2.805, 3.1313), (-3.779, -3.283), (3.584, -1.848)]


fig, axes = plt.subplots(1, 4, figsize=(12, 4), dpi=256)
axes = axes.ravel()

y = -agent._get_task_fitness(0)

post = agent.model.posterior(xg)
pred = -post.mean[..., 0].detach()
err = post.variance[..., 0].detach()

norm = mpl.colors.Normalize(-500, 0)

cmap = "magma"

zoom = 8

uxg1 = sp.ndimage.zoom(xg[..., 0], zoom=zoom)
uxg2 = sp.ndimage.zoom(xg[..., 1], zoom=zoom)
umu = sp.ndimage.zoom(pred, zoom=zoom)
ulv = sp.ndimage.zoom(np.log(err.sqrt()), zoom=zoom)
uobj = sp.ndimage.zoom(obj_grid, zoom=zoom)

samp = axes[0].scatter(*agent.active_inputs.values.T, c=-y, s=16, norm=norm, cmap=cmap)
# axes[1].pcolormesh(uxg1, uxg2, -umu, norm=norm, cmap=cmap)
# err = axes[2].pcolormesh(uxg1, uxg2, ulv, cmap=cmap)
# acq = axes[3].pcolormesh(uxg1, uxg2, uobj, cmap=cmap)


mean = axes[1].imshow(-umu.T[::-1], norm=norm, extent=[-6, 6, -6, 6], cmap=cmap, aspect="auto")
lvar = axes[2].imshow(ulv.T[::-1], extent=[-6, 6, -6, 6], cmap=cmap, aspect="auto")
acqf = axes[3].imshow(uobj.T[::-1], extent=[-6, 6, -6, 6], cmap=cmap, aspect="auto")

for ax in axes:
    for x, y in optima:
        ax.scatter(x, y, facecolor="none", edgecolor="k", marker="o", lw=5e-1, s=16)

    ax.set_xlim(-6, 6)
    ax.set_ylim(-6, 6)
    ax.set_xticks(np.arange(-6, 7, 2))
    ax.set_yticks(np.arange(-6, 7, 2))
    ax.set_xlabel("$x_1$")
    ax.set_ylabel("$x_2$")

axes[3].scatter(*X.T, marker="x", lw=5e-1, color="k", s=16)

# axes[0].set_title("sampled points")
# axes[1].set_title("posterior mean")
# axes[2].set_title("posterior log std. dev.")
# axes[3].set_title("$q$-expected improvement")


# clb1 = fig.colorbar(mappable=mpl.cm.ScalarMappable(norm=norm), ax=axes[0:2], location="bottom", shrink=0.8, aspect=32)
clb1 = fig.colorbar(samp, ax=axes[0], location="bottom", shrink=0.8, aspect=32)
clb1.set_label("sampled points")
clb2 = fig.colorbar(mean, ax=axes[1], location="bottom", shrink=0.8, aspect=32)
clb2.set_label("posterior mean")
clb3 = fig.colorbar(lvar, ax=axes[2], location="bottom", shrink=0.8, aspect=32)
clb3.set_label("posterior log std. dev.")
clb4 = fig.colorbar(acqf, ax=axes[3], location="bottom", shrink=0.8, aspect=32)
clb4.set_label("$q$-expected improvement")
# clb3 = fig.colorbar(acq, ax=axes[3], location="bottom", shrink=0.8, aspect=32)

plt.tight_layout()
plt.savefig("../plots/bayesian_himmelblau.pdf", bbox_inches="tight", transparent=True, pad_inches=0, dpi=256)

In [ ]:
np.exp(ulv)

In [ ]:
plt.imshow(obj)